In [1]:
# Import des librairies
import numpy as np
import pandas as pd

from pandas import Series,DataFrame

# Question 15 #
* création d'une table appelée population, contenant la population de chaque pays pour chaque année comprise entre 2012 et la dernière année disponible de la FAO
* Elle devra contenir 4 colonnes : pays, code_pays, annee, population (Je préfère ctry_cd, country year, population)
* Proposez une clé primaire pertinente pour cette table.

**NB :** la dernière année étant 2013, j'ai choisi d'importer 2010-2013 pour avoir 4 années

In [2]:
# Import des données 'db_population 2010-2013' de la FAO
pop = pd.read_csv('data/db_population 2010-2013.csv')
pop.head(1)

,Domain Code,Domain,Country Code,Country,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,FBS,Food Balance Sheets,2,Afghanistan,511,Total Population - Both sexes,2501,Population,2010,2010,1000 persons,28398,NaN,Official data


### 15.1 - data wrangling, création du dataframe

In [3]:
# supression des variables inutiles
pop.drop(['Domain Code', 'Domain', 'Element', 'Element Code', 'Item Code', 'Item', 
          'Year Code', 'Unit', 'Flag','Flag Description'], axis=1, inplace = True)

# On renomme les variables de pop
pop.columns = ['ctry_cd','country','year','population']

# on multiplie 'population' par sa valeur = 1000 persons
pop['population'] = pop['population'] * 1000

population = pop.copy()
population.head(1)

,ctry_cd,country,year,population
0,2,Afghanistan,2010,28398000


### 15.2 - problème de l'agrégat 'China'
3 solutions sont possibles:
* on conserve le df 'population' en l'état tout en sachant qu'il contient un doublon, l'agrégat 'China' (ctry_cd 351)
* on exclut les pays qui constituent l'agrégat 'China' = 96 + 128 + 41 + 214
* on exclut l'agrégat 'China' = 351

Je choisi de garder 'population' dans son entier en gardant en mémoire l'agrégat pour avoir plus de choix

In [4]:
# on exclut les pays qui constituent l'agrégat 'China' => ctry_cd 351 = 96 + 128 + 41 + 214
popu1 = population[~(population['ctry_cd'].isin([96, 128, 41, 214]))]
popu1.head()

,ctry_cd,country,year,population
0,2,Afghanistan,2010,28398000
1,2,Afghanistan,2011,29105000
2,2,Afghanistan,2012,29825000
3,2,Afghanistan,2013,30552000
4,3,Albania,2010,3150000


In [5]:
# on choisit les pays qui constituent l'agrégat 'China' => ctry_cd 351 = 96 + 128 + 41 + 214
popu2 = population[(population['ctry_cd'].isin([96, 128, 41, 214]))]
popu2

,ctry_cd,country,year,population
128,96,"China, Hong Kong SAR",2010,7050000
129,96,"China, Hong Kong SAR",2011,7096000
130,96,"China, Hong Kong SAR",2012,7148000
131,96,"China, Hong Kong SAR",2013,7204000
132,128,"China, Macao SAR",2010,535000
133,128,"China, Macao SAR",2011,546000
134,128,"China, Macao SAR",2012,557000
135,128,"China, Macao SAR",2013,566000
136,41,"China, mainland",2010,1359821000
137,41,"China, mainland",2011,1368440000


### 15.3 - La clé primaire pertinente est [ctry_cd, year]
* le country code n'est pas unique, il dépend aussi des valeurs de la variable 'year'
* Dans SQL la PK doit être unique, il faudrait rajouter une colonne pour concaténer les valeurs pour avoir une clé unique
* Mais la méthode GROUP BY de SQL nous permet de faire la même chose sans avoir à créer une colonne concatenate

In [6]:
# Export au format .csv - Attention ici l'agregat 'China' est inclus
population.to_csv('data/population.csv', index = False)

# Question 16 # 
Créez une table appelée **dispo_alim** contenant pour **chaque pays**, pour **chaque produit**, et pour **chaque année** comprise entre 2012 et la dernière année disponible, les informations suivantes:
* la nature du produit (deux valeurs possibles : “animal” ou “végétal”)
* disponibilité alimentaire en tonnes
* disponibilité alimentaire en Kcal/personne/jour
* disponibilité alimentaire de protéines en g/personne/jour
* disponibilité alimentaire de matières grasses en g/personne/jour

**NB :** la dernière année étant 2013, j'ai choisi d'importer 2010-2013 pour avoir 4 années

### 16.1 - Création de la table végétal/animal par années

In [7]:
# Import des bilans alimentaires 'vegetal' 2010-2013 (impossible en une fois, fichier trop volumineux pour la db de la FAO)
veg1 = pd.read_csv('data/db_vegetal 2010-2011.csv')
veg2 = pd.read_csv('data/db_vegetal 2012-2013.csv')
# On fait l'union de 'veg1' et de 'veg2'
veg = veg1.append(veg2)
# check 'Year'
veg['Year'].unique()

array([2010, 2011, 2012, 2013], dtype=int64)

In [8]:
# Import des bilans alimentaires 'animal' 2010-2013 (impossible en une fois, fichier trop volumineux pour la db de la FAO)
ani1 = pd.read_csv('data/db_animal 2010-2011.csv')
ani2 = pd.read_csv('data/db_animal 2012-2013.csv')
# On fait l'union de 'veg1' et de 'veg2'
ani = ani1.append(ani2)
# check 'Year'
ani['Year'].unique()

array([2010, 2011, 2012, 2013], dtype=int64)

In [9]:
# Ajout de la variable origin
veg['origin'] = 'vegetal'
ani['origin'] = 'animal'

# On fait l'union de 'veg' et de 'ani' dans une nouvelle variable
ori = ani.append(veg)

# On garde les variables utiles mais aussi 'Element' nécessaire pour la 2e série de variables (pivot)
ori.drop(['Domain Code', 'Domain', 'Element Code', 'Year Code','Unit','Flag', 'Flag Description'], axis=1, inplace = True)
ori.head(1)

ori.head(2)

,Country Code,Country,Element,Item Code,Item,Year,Value,origin
0,2,Afghanistan,Production,2731,Bovine Meat,2010,131.0,animal
1,2,Afghanistan,Production,2731,Bovine Meat,2011,138.0,animal


### 16.2 - Elle devra contenir ces colonnes 
* code_pays = ctry_cd
* pays = country
* année = year
* code_produit = item_cd
* produit = item
* origin = origin

**NB :** Je retire la variable 'Unit' qui influe sur la colonne 'Elements', mais nous savons que certaines valeurs de cette colonne contiennent déjà leurs unités. Pour les autres (ex: 'Food'), elles sont expirmées en 1000 tonnes, il faudra conc multiplier par 1000 ces valeurs une fois le pivot effectué fin d'avoir un résultat en 'tonnes'

In [10]:
# On renomme les colonnes
ori.columns = ['ctry_cd','country', 'element', 'item_cd', 'item','year','value', 'origin']
ori.head(1)

,ctry_cd,country,element,item_cd,item,year,value,origin
0,2,Afghanistan,Production,2731,Bovine Meat,2010,131.0,animal


### 16.3 - Réalisation du pivot de la colonne 'element'

**NB :** cette étape est importante car une fois effectué le df 'pivot' nous aurons à nous en servir plusieurs fois dans les prochaines étapes

In [11]:
# valeurs uniques de 'element'
ori['element'].unique()

array(['Production', 'Import Quantity', 'Domestic supply quantity',
       'Food', 'Food supply quantity (kg/capita/yr)',
       'Food supply (kcal/capita/day)',
       'Protein supply quantity (g/capita/day)',
       'Fat supply quantity (g/capita/day)', 'Seed', 'Losses',
       'Export Quantity', 'Feed', 'Other uses', 'Stock Variation',
       'Processing'], dtype=object)

In [12]:
# pivot de la variable 'element'
pivot = ori.pivot_table(
    index=['ctry_cd','country', 'item_cd','item', 'year','origin'],
    columns = ['element'], values=['value'], aggfunc=sum)
pivot.columns = ['Domestic supply quantity','Export Quantity','Fat supply quantity (g/capita/day)',
                 'Feed','Food','Food supply (kcal/capita/day)','Food supply quantity (kg/capita/yr)',
                 'Import Quantity','Losses','Other uses','Processing','Production',
                 'Protein supply quantity (g/capita/day)','Seed','Stock Variation']
pivot = pivot.reset_index()
pivot.head(1)

,ctry_cd,country,item_cd,item,year,origin,Domestic supply quantity,Export Quantity,Fat supply quantity (g/capita/day),Feed,...,Food supply (kcal/capita/day),Food supply quantity (kg/capita/yr),Import Quantity,Losses,Other uses,Processing,Production,Protein supply quantity (g/capita/day),Seed,Stock Variation
0,1,Armenia,2511,Wheat and products,2010,vegetal,505.0,0.0,3.46,61.0,...,978.0,125.41,353.0,38.0,1.0,12.0,183.0,29.15,22.0,-31.0


* Comme dit ci-dessus, le df 'pivot' servira plusieurs fois plus tard
* Il faut garder en mémoire que les variables ne comportant pas d'unités de mesure devoint être multipliées par 1000 pour être exprimées en tonnes

### 16.4 - ...Et celles-ci, issues d'un pivot de la colonne 'Element'
* dispo_alim_tonnes  =  food_t_ctry_y
* dispo_alim_kcal_pj  =  food_kcal_cap_y
* dispo_prot  =  protein_g_cap_d
* dispo_mat_gr =  fat_g_cap_d


In [13]:
# on sélectionne les valeurs requises de l'ex-colonne 'element'
dispo = pivot.copy()
dispo.drop(['Domestic supply quantity','Export Quantity','Feed','Food supply quantity (kg/capita/yr)','Import Quantity',
            'Losses','Other uses','Processing','Production','Seed','Stock Variation'], axis = 1, inplace = True)
dispo.head(2)

,ctry_cd,country,item_cd,item,year,origin,Fat supply quantity (g/capita/day),Food,Food supply (kcal/capita/day),Protein supply quantity (g/capita/day)
0,1,Armenia,2511,Wheat and products,2010,vegetal,3.46,372.0,978.0,29.15
1,1,Armenia,2511,Wheat and products,2011,vegetal,3.58,386.0,1016.0,30.26


In [14]:
# On renomme les colonnes
dispo.columns = ['ctry_cd','country','item_cd','item','year','origin','fat_g_cap_d','food_t_ctry_y',
                 'food_kcal_cap_d','protein_g_cap_d']
# on multiplie 'food (t/ctry/y) par 1000
dispo['food_t_ctry_y'] = dispo['food_t_ctry_y'] * 1000
dispo.head(2)

,ctry_cd,country,item_cd,item,year,origin,fat_g_cap_d,food_t_ctry_y,food_kcal_cap_d,protein_g_cap_d
0,1,Armenia,2511,Wheat and products,2010,vegetal,3.46,372000.0,978.0,29.15
1,1,Armenia,2511,Wheat and products,2011,vegetal,3.58,386000.0,1016.0,30.26


### 16.5 - Proposez une clé primaire pertinente pour cette table
* La clé primaire pertinente est [ctry_cd,item_cd,year]

In [15]:
# Export au format .csv
dispo.to_csv('data/dispo_alim.csv', index = False)

# Question 17
Une table appelée **equilibre_prod** contenant pour **chaque pays**, pour **chaque produit**, et pour **chaque année** comprise entre 2012 et la dernière année disponible, les quantités suivantes :
* disponibilité intérieure, aliments pour animaux, semences, pertes, transformés, nourriture, autres utilisations

Elle devra contenir ces colonnes : 
* pays, code_pays, année, produit, code_produit, dispo_int, alim_ani, semences, pertes, transfo, nourriture, autres_utilisations

Proposez une clé primaire pertinente pour cette table.

**NB :** une fois encore je renomme les colonnes à ma façon que je trouve plus explicite

### 17.1 - data wrangling, création du dataframe

In [16]:
# on repart du df 'pivot' de l'exercice 16.3
equil=pivot.copy()
equil.head(1)

,ctry_cd,country,item_cd,item,year,origin,Domestic supply quantity,Export Quantity,Fat supply quantity (g/capita/day),Feed,...,Food supply (kcal/capita/day),Food supply quantity (kg/capita/yr),Import Quantity,Losses,Other uses,Processing,Production,Protein supply quantity (g/capita/day),Seed,Stock Variation
0,1,Armenia,2511,Wheat and products,2010,vegetal,505.0,0.0,3.46,61.0,...,978.0,125.41,353.0,38.0,1.0,12.0,183.0,29.15,22.0,-31.0


In [17]:
# on sélectionne les valeurs requises par l'exercice
equil.drop(['origin','Export Quantity','Import Quantity','Production','Stock Variation',
            'Food supply quantity (kg/capita/yr)','Fat supply quantity (g/capita/day)',
            'Food supply (kcal/capita/day)','Protein supply quantity (g/capita/day)'], axis=1, inplace = True)
equil.head(2)

,ctry_cd,country,item_cd,item,year,Domestic supply quantity,Feed,Food,Losses,Other uses,Processing,Seed
0,1,Armenia,2511,Wheat and products,2010,505.0,61.0,372.0,38.0,1.0,12.0,22.0
1,1,Armenia,2511,Wheat and products,2011,516.0,65.0,386.0,34.0,0.0,12.0,20.0


In [18]:
# On renomme les colonnes
equil.columns = ['ctry_cd','country','item_cd','item','year','domestic_t_ctry_y','feed_t_ctry_y',
                 'food_t_ctry_y','losses_t_ctry_y','other_uses_t_ctry_y','processing_t_ctry_y','seeds_t_ctry_y']


# on multiplie par 1000 les colonnes nécessaires
equil['domestic_t_ctry_y'] = equil['domestic_t_ctry_y'] * 1000
equil['feed_t_ctry_y'] = equil['feed_t_ctry_y'] * 1000
equil['food_t_ctry_y'] = equil['food_t_ctry_y'] * 1000
equil['losses_t_ctry_y'] = equil['losses_t_ctry_y'] * 1000
equil['other_uses_t_ctry_y'] = equil['other_uses_t_ctry_y'] * 1000
equil['processing_t_ctry_y'] = equil['processing_t_ctry_y'] * 1000
equil['seeds_t_ctry_y'] = equil['seeds_t_ctry_y'] * 1000
equil.head(2)

,ctry_cd,country,item_cd,item,year,domestic_t_ctry_y,feed_t_ctry_y,food_t_ctry_y,losses_t_ctry_y,other_uses_t_ctry_y,processing_t_ctry_y,seeds_t_ctry_y
0,1,Armenia,2511,Wheat and products,2010,505000.0,61000.0,372000.0,38000.0,1000.0,12000.0,22000.0
1,1,Armenia,2511,Wheat and products,2011,516000.0,65000.0,386000.0,34000.0,0.0,12000.0,20000.0


### 17.2 - Proposez une clé primaire pertinente pour cette table
* La clé primaire pertinente est [ctry_cd,item_cd,year]

In [19]:
# Export au format .csv
equil.to_csv('data/equilibre_prod.csv', index = False)

# Question 18
* Création d'une table appelée **sous_nutrition**, contenant le nombre de personnes en sous-alimentation pour chaque pays et pour chaque année comprise entre 2012 et la dernière année disponible. 
* Elle devra contenir 4 colonnes : pays, code_pays, année, nb_personnes.
* proposez une clé primaire pertinente pour cette table

In [20]:
# Import des données 'db_undernourished 2012-2016' de la FAO
undnr = pd.read_csv('data/db_undernourished 2012-2016.csv')
undnr.head(1)

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,FS,Suite of Food Security Indicators,2,Afghanistan,6132,Value,210011,Number of people undernourished (million) (3-y...,20112013,2011-2013,millions,7.2,F,FAO estimate


### 18.1 - data wrangling

In [21]:
# On retire les variables inutiles
undnr.drop(['Domain Code', 'Domain','Element Code','Element','Item Code',
            'Item','Unit','Year Code','Flag', 'Flag Description'], axis=1, inplace = True)
undnr.head(1)

,Area Code,Area,Year,Value
0,2,Afghanistan,2011-2013,7.2


In [22]:
# on retire les éventuels NaN
undnr = undnr[undnr['Value'].notnull()]
undnr['Value'].count()

835

In [23]:
# format des variables
undnr.dtypes

Area Code     int64
Area         object
Year         object
Value        object
dtype: object

In [24]:
# transformation de 'Value' en float64
undnr['Value'] = pd.to_numeric(undnr.Value, errors='coerce')
# format de 'Value'
undnr.dtypes

Area Code      int64
Area          object
Year          object
Value        float64
dtype: object

In [25]:
# on multiplie 'Value' par sa valeur = 1000000 persons
undnr['Value'] = undnr['Value'] * 1000000
undnr.head(1)

,Area Code,Area,Year,Value
0,2,Afghanistan,2011-2013,7200000.0


In [26]:
# Valeurs uniques de 'Year'
undnr['Year'].unique()

array(['2011-2013', '2012-2014', '2013-2015', '2014-2016', '2015-2017'],
      dtype=object)

In [27]:
# Les valeurs étant la moyenne sur 3 ans, on les remplace par la valeur moyenne
undnr['Year'].replace('2011-2013','2012', inplace = True)
undnr['Year'].replace('2012-2014','2013', inplace = True)
undnr['Year'].replace('2013-2015','2014', inplace = True)
undnr['Year'].replace('2014-2016','2015', inplace = True)
undnr['Year'].replace('2015-2017','2016', inplace = True)
undnr['Year'].unique()

array(['2012', '2013', '2014', '2015', '2016'], dtype=object)

In [28]:
# On renomme les variables
undnr.columns = ['ctry_cd','country','year','undernourished']
undnr.head(1)

,ctry_cd,country,year,undernourished
0,2,Afghanistan,2012,7200000.0


### 18.2 - Proposez une clé primaire pertinente
* La clé primaire pertinente est [ctry_cd,year]